***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE  SCHEDULED_STOP
cluster-0016  GCE       2                                             RUNNING  us-central1-a


To take a quick anonymous survey, run:
  $ gcloud survey



# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Dec  8 19:12 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [11]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'shay-208886382-bucket'
full_path = f"gs://{bucket_name}/"
paths = []

client = storage.Client()
blobs = client.list_blobs(bucket_name)

for b in blobs:
    # keep ONLY parquet files (critical)
    if b.name.endswith(".parquet"):
        paths.append(full_path + b.name)

print("Number of parquet files:", len(paths))
print("Sample paths:", paths[:5])


Number of parquet files: 60
Sample paths: ['gs://shay-208886382-bucket/multistream10_preprocessed.parquet', 'gs://shay-208886382-bucket/multistream11_part2_preprocessed.parquet', 'gs://shay-208886382-bucket/multistream11_preprocessed.parquet', 'gs://shay-208886382-bucket/multistream12_part2_preprocessed.parquet', 'gs://shay-208886382-bucket/multistream12_preprocessed.parquet']


***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [13]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd


In [14]:

# 1) build doc_id -> title dictionary
doc_title_dic = (
    parquetFile
    .select("id", "title")
    .rdd
    .map(lambda r: (int(r["id"]), r["title"]))
    .collectAsMap()
)

# 2) save locally
with open("doc_title_dic.pickle", "wb") as f:
    pickle.dump(doc_title_dic, f, protocol=pickle.HIGHEST_PROTOCOL)

# 3) upload to GCS bucket root
client = storage.Client()
bucket = client.bucket(bucket_name)
bucket.blob("doc_title_dic.pickle").upload_from_filename("doc_title_dic.pickle")

print("uploaded doc_title_dic.pickle, size:", len(doc_title_dic))


uploaded doc_title_dic.pickle, size: 6348910


We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [9]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [10]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [11]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [12]:
from inverted_index_gcp import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [19]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124

def token2bucket_id(token):
    return int(_hash(token), 16) % NUM_BUCKETS

from collections import Counter
from inverted_index_gcp import InvertedIndex


def word_count(text, doc_id):
    if text is None:
        return []
    if not isinstance(text, str):
        text = str(text)

    tokens = [m.group() for m in RE_WORD.finditer(text.lower())]
    tokens = [t for t in tokens if t not in all_stopwords]

    w2cnt = Counter(tokens)
    return [(w, (doc_id, tf)) for w, tf in w2cnt.items()]


def reduce_word_counts(unsorted_pl):
    # unsorted_pl: iterable of (doc_id, tf)
    return sorted(unsorted_pl, key=lambda x: x[0])


def calculate_df(postings_rdd):
    # postings_rdd: (term, posting_list)
    return postings_rdd.map(lambda x: (x[0], len(x[1])))


def partition_postings_and_write(postings_rdd):
    # postings_rdd: (term, posting_list)
    buckets = postings_rdd.map(lambda x: (token2bucket_id(x[0]), x))
    grouped = buckets.groupByKey()

    def write_bucket(bucket_tuple):
        bucket_id, iterable_items = bucket_tuple
        list_w_pl = list(iterable_items)
        b_w_pl = (bucket_id, list_w_pl)
        posting_locs = InvertedIndex.write_a_posting_list(b_w_pl, bucket_name)
        return posting_locs

    return grouped.map(write_bucket)


In [ ]:
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered).collect()
index_const_time = time() - t_start

In [21]:
# test index construction time
assert index_const_time < 60*120

In [2]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

NameError: name 'defaultdict' is not defined

In [1]:
import math

# 1. הגדרת N (מספר הדפים שספרת בתא 9)
N_docs = 6348910

# 2. חישוב IDF לכל מילה (נשתמש ב-w2df_dict שיצרת בתא 20)
# log10(N/df) הוא הסטנדרט המקובל
idf_dict = {word: math.log10(N_docs / df) for word, df in w2df_dict.items()}

# 3. חישוב נורמה לכל מסמך (doc_norm) לטובת קוסינוס סימילריטי
# עוברים על ה-postings_filtered, מחשבים (tf * idf)^2 ומחברים לכל מסמך
doc_norms = postings_filtered.flatMap(lambda x: [(doc_id, (tf * idf_dict.get(x[0], 0))**2) for doc_id, tf in x[1]]) \
                             .reduceByKey(lambda a, b: a + b) \
                             .mapValues(lambda s: math.sqrt(s))

# 4. חישוב אורך כל מסמך (doc_len)
# נשתמש ב-doc_text_pairs מתא 8 ובפונקציית ה-word_count מתא 19
doc_lens = doc_text_pairs.map(lambda x: (x[1], len(word_count(x[0], x[1]))))

# 5. איחוד לתוך מילון weights: {doc_id: (len, norm)}
full_weights_dict = doc_lens.leftOuterJoin(doc_norms) \
                            .mapValues(lambda x: (x[0], x[1] if x[1] is not None else 0.0)) \
                            .collectAsMap()

NameError: name 'w2df_dict' is not defined

Putting it all together

In [23]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict

inverted.N = N_docs
inverted.weights = full_weights_dict

# write the global stats out
inverted.write_index('.', 'index')
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 18.4 MiB/ 18.4 MiB]                                                
Operation completed over 1 objects/18.4 MiB.                                     


In [24]:
!gsutil ls -lh $index_dst

 18.45 MiB  2025-12-09T13:22:30Z  gs://shay-208886382-bucket/postings_gcp/index.pkl
TOTAL: 1 objects, 19342504 bytes (18.45 MiB)


# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [25]:
def generate_graph(pages):
  edges = (
        pages.flatMap(
            lambda row: [
                (row.id, anchor[0])       
                for anchor in (row.anchor_text or [])
                if anchor is not None and anchor[0] is not None
            ]
        )
        .distinct()
    )


  vertices = (
        edges
        .flatMap(lambda e: e)     
        .distinct()
        .map(lambda vid: (vid,))  
    )
  return edges, vertices

In [ ]:
t_start = time()
pages_links = (
    spark.read
         .parquet(f"gs://{bucket_name}/multistream*")
         .select("id", "anchor_text")
         .rdd
)
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

+-------+------------------+
|     id|          pagerank|
+-------+------------------+
|3434750| 9913.728782160775|
|  10568|  5385.34926364204|
|  32927| 5282.081575765277|
|  30680| 5128.233709604118|
|5843419| 4957.567686263866|
|  68253|4769.2782653551585|
|  31717| 4486.350180548309|
|  11867|  4146.41465091277|
|  14533| 3996.466440885503|
| 645042|3531.6270898037437|
|  17867| 3246.098390604142|
|5042916|2991.9457391661786|
|4689264| 2982.324883041747|
|  14532|2934.7468292031695|
|  25391| 2903.546223513398|
|   5405|2891.4163291546365|
|4764461|2834.3669873326617|
|  15573| 2783.865118158838|
|   9316| 2782.039646413769|
|8569916|2775.2861918400163|
+-------+------------------+
only showing top 20 rows



In [27]:
# test that PageRank computaion took less than 1 hour
assert pr_time < 60*120

# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete 
the reporting requirements for assignment #3. 

In [28]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

14.28 GiB    gs://wikidata_preprocessed


In [29]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_gcp/'
!gsutil du -sh "$index_dst"

5.92 GiB     gs://shay-208886382-bucket/postings_gcp


In [31]:
# How many USD credits did you use in GCP during the course of this assignment?
cost = 2.54 
print(f'I used {cost} USD credit during the course of this assignment')

I used 2.52 USD credit during the course of this assignment


**Bonus (10 points)** if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour. 


In [ ]:
#If you have decided to do the bonus task - please copy the code here 

bonus_flag = False # Turn flag on (True) if you have implemented this part

t_start = time()

pages_links = (
    spark.read.parquet(f"gs://{bucket_name}/multistream*")
         .select("id", "anchor_text")
         .rdd
)
edges = edges.distinct().cache()          
vertices = vertices.distinct().cache()    

links = (
    edges
    .groupByKey()
    .mapValues(list)   
    .cache()
)

num_vertices = vertices.count()

ranks = vertices.map(lambda v: (v[0], 1.0 / num_vertices)).cache()

d = 0.85  

for it in range(10):

    contribs = (
        links
        .join(ranks)  
        .flatMap(
            lambda node_links_rank: [
                (dst, node_links_rank[1][1] / len(node_links_rank[1][0]))
                for dst in node_links_rank[1][0]
                if len(node_links_rank[1][0]) > 0
            ]
        )
    )

    contribs_sum = contribs.reduceByKey(lambda x, y: x + y)

    dangling_mass = (
        ranks
        .subtractByKey(links)  
        .values()
        .sum()
    )

    base = (1.0 - d) / num_vertices
    dangling_contrib = d * dangling_mass / num_vertices

    new_ranks_partial = contribs_sum.mapValues(
        lambda contrib: base + d * contrib + dangling_contrib
    )

    ranks = (
        vertices
        .map(lambda v: (v[0], 0.0))          
        .leftOuterJoin(new_ranks_partial)    
        .mapValues(
            lambda x: x[1] if x[1] is not None else base + dangling_contrib
        )
        .cache()
    )


pr_bonus_df = ranks.toDF(["id", "pagerank"])
pr_bonus_df = pr_bonus_df.orderBy(col("pagerank").desc())

pr_bonus_df.repartition(1).write.csv(
    f"gs://{bucket_name}/pr_bonus",
    compression="gzip"
)

pr_time_Bonus = time() - t_start


25/12/09 15:30:35 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_196_40 !
25/12/09 15:30:35 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_196_30 !
25/12/09 15:30:35 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_186_53 !
25/12/09 15:30:35 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_104_42 !
25/12/09 15:30:35 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_196_1 !
25/12/09 15:30:35 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_196_22 !
25/12/09 15:30:35 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_89_36 !
25/12/09 15:30:35 WARN org.apache.spark.storage.BlockManagerMasterEndpoint: No more replicas available for rdd_89_115 !
25/12/09 15:30:35 WARN org.apache.spark.st

KeyboardInterrupt: 

In [33]:
# Note:test that PageRank computaion took less than 1 hour
assert pr_time_Bonus < 60*60 and bonus_flag

NameError: name 'pr_time_Bonus' is not defined

25/12/09 16:33:40 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 2200.0 in stage 590.0 (TID 72926) (cluster-0016-w-0.us-central1-a.c.ferrous-module-480122-e7.internal executor 12): TaskKilled (Stage cancelled)
25/12/09 16:33:40 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 2201.0 in stage 590.0 (TID 72927) (cluster-0016-w-1.us-central1-a.c.ferrous-module-480122-e7.internal executor 13): TaskKilled (Stage cancelled)
25/12/09 16:33:40 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 2203.0 in stage 590.0 (TID 72929) (cluster-0016-w-1.us-central1-a.c.ferrous-module-480122-e7.internal executor 13): TaskKilled (Stage cancelled)
25/12/09 16:33:40 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 2204.0 in stage 590.0 (TID 72930) (cluster-0016-w-1.us-central1-a.c.ferrous-module-480122-e7.internal executor 15): TaskKilled (Stage cancelled)
25/12/09 16:33:40 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 2202.0 in stage 590.0 (TID 72928) (clust